### Steady State

#### Inclined Slider Use case

- non-periodic
- density boundary conditions left and right
- no energy included

To activate FEM:

```yaml
numerics:
    solver: fem
```

To select the terms to include (see `Equations` section in [FEM theory](./08_FEM1d_theory.ipynb)):

```yaml
fem_solver:
    equations:
        term_list: ['R11', 'R11S', 'R21', 'R24']
```

When solving steady state, an internal convergence loop is used.

In [ ]:
# non-periodic boundary conditions and steady-state; inclined slider use case
%matplotlib inline
from GaPFlow import Problem

sim = """
options:
    output: data/inclined
    write_freq: 1000
    silent: False
grid:
    Lx: 0.1
    Ly: 1.
    Nx: 100
    Ny: 1
    xE: ['D', 'N', 'N']
    xW: ['D', 'N', 'N']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
    xE_D: 1.1853
    xW_D: 1.1853
geometry:
    type: inclined
    hmax: 6.6e-5
    hmin: 1.0e-5
    U: 50.
    V: 0.
numerics:
    solver: fem
    tol: 1e-6
    dt: 1e-8
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    equations:
        term_list: ['R11', 'R11S', 'R21', 'R24']
"""

problem = Problem.from_string(sim)
problem.run()
problem.plot()

### Dynamic

#### Journal Use Case

Activate dynamic mode (default is False):

```yaml
fem_solver:
    dynamic: True
```

Note that in the dynamic case

```yaml
numerics:
    tol: 1e-8
    max_it: 5
```

applies to the problem main loop and

```yaml
fem_solver:
    R_norm_tol: 1e-04
    max_iter: 10
```

applies to the inner loop that needs to converge for each time step.

In [ ]:
# periodic boundary conditions and time-stepping; journal use case
%matplotlib inline
from GaPFlow import Problem

sim = """
options:
    output: data/journal
    write_freq: 1
    silent: False
grid:
    dx: 1.e-5
    dy: 1.
    Nx: 100
    Ny: 1
    xE: ['P', 'P', 'P']
    xW: ['P', 'P', 'P']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
geometry:
    type: journal
    CR: 1.e-2
    eps: 0.7
    U: 0.1
    V: 0.
numerics:
    solver: fem
    tol: 1e-8
    dt: 1e-02
    max_it: 5
properties:
    shear: 0.0794
    bulk: 0.
    EOS: DH
    P0: 101325.
    rho0: 877.7007
    T0: 323.15
    C1: 3.5e10
    C2: 1.23
fem_solver:
    type: newton_alpha
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-04
    equations:
        term_list: ['R11', 'R11S', 'R21', 'R24', 'R1T', 'R2T']
"""

problem = Problem.from_string(sim)
problem.run()
problem.plot()

### Energy

#### Shear Wall Heating

Dynamic simulation of Couette flow including the *wall stress work* term from the `energy` equations.

Observe how the mean total energy (and temperature) rises due to work done by the movement of the wall.

Inclusion of the wall stress and time derivative terms of the energy equation. Note that all other energy terms such as heat exchange with the wall are not included here.

```yaml
['R34', 'R3T']
```

Relevant for this case is also the volume-specific heat capacity $c_v$, which determines the temperature increase.

```yaml
energy_spec:
    cv: 1005.
```


In [ ]:
# shear wall heating
%matplotlib inline
from GaPFlow import Problem

sim = """
options:
    output: data/journal
    write_freq: 1
    silent: False
grid:
    dx: 1.e-5
    dy: 1.
    Nx: 100
    Ny: 1
    xE: ['P', 'P', 'P']
    xW: ['P', 'P', 'P']
    yS: ['P', 'P', 'P']
    yN: ['P', 'P', 'P']
geometry:
    type: inclined
    hmax: 1.0e-5
    hmin: 1.0e-5
    U: 1.
    V: 0.
numerics:
    solver: fem
    tol: 0
    dt: 1e-5
    max_it: 20
properties:
    EOS: PL
    shear: 1.846e-5
    bulk: 0.
    P0: 101325
    rho0: 1.1853
    alpha: 0.
fem_solver:
    dynamic: True
    max_iter: 10
    R_norm_tol: 1e-04
    equations:
        energy: True
        term_list: ['R11', 'R11S', 'R1T', 'R21', 'R24', 'R2T', 'R34', 'R3T']
energy_spec:
    cv: 1005.
"""
import numpy as np

E_mean_history = []
T_mean_history = []

def compute_means(args):
    E_mean = np.mean(problem.energy.energy)
    E_mean_history.append(E_mean)
    T_mean = np.mean(problem.energy.T_quad(2))
    T_mean_history.append(T_mean)

problem = Problem.from_string(sim)
problem.solver.num_solver.callback_fun = compute_means
problem.run()
problem.plot()

In [ ]:
problem.animate()